In [1]:
!date

Mon Jun 14 10:30:17 PDT 2021


Import all packages (some not necessary)

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

Load in 10x sequencing data.

In [5]:
# Data from hEP-structures
Goetz_D = sc.read_10x_mtx("MZG_data/Goetz-3_hEP")

# Data from D5/D6 natural embryos
Goetz_D2 = sc.read_10x_mtx("MZG_data/Goetz-4_natural")

Concatenate the two data frames. 

In [6]:
adata = Goetz_D.concatenate(Goetz_D2, index_unique=None)

Organize the data.

In [7]:
# Make gene name column
adata.var["gene_name"] = adata.var.index
adata.var

,gene_ids,feature_types,gene_name
RP11-34P13.3,ENSG00000243485,Gene Expression,RP11-34P13.3
FAM138A,ENSG00000237613,Gene Expression,FAM138A
OR4F5,ENSG00000186092,Gene Expression,OR4F5
RP11-34P13.7,ENSG00000238009,Gene Expression,RP11-34P13.7
RP11-34P13.8,ENSG00000239945,Gene Expression,RP11-34P13.8
...,...,...,...
AC233755.2,ENSG00000277856,Gene Expression,AC233755.2
AC233755.1,ENSG00000275063,Gene Expression,AC233755.1
AC240274.1,ENSG00000271254,Gene Expression,AC240274.1
AC213203.1,ENSG00000277475,Gene Expression,AC213203.1


Load in meta data (note: analysis was done in the other notebook in order to generate this meta data file, and then was added retrospectively.)

In [9]:
# Read in meta data csv
all_meta = pd.read_csv("MZG_data/all_meta.csv", index_col=0)

# Generate 'cell_group' column to distinguish natural embryo, 2D EPSCs, D5 hEP, and D6 hEP.
# Note that 'sample_id' distinguishes between replicates.
all_meta['cell_group'] = [all_meta['sample_id'][i].split('-')[0] for i in range(len(all_meta))]
all_meta.index.name = None

all_meta

,sample_id,sample_number,cell_group
AAACCCAAGCGATGAC11,unknown,0,unknown
AAACCCAAGGTAAAGG11,D6-R3,6,D6
AAACCCAAGTTGAAGT11,D6-R1,4,D6
AAACCCACAAATCCCA11,D6-R3,6,D6
AAACCCACAACCGTGC11,D6-R1,4,D6
...,...,...,...
TTTCACAGTCCGGACT-1,Nat,9,Nat
TTTCATGCATCCTCAC-1,Nat,9,Nat
TTTGATCAGGAGCAAA-1,Nat,9,Nat
TTTGGTTAGACGGTTG-1,Nat,9,Nat


In [10]:
# Add meta data to adata.obs
adata.obs =pd.concat ([adata.obs, all_meta], axis=1)

adata.obs

,batch,sample_id,sample_number,cell_group
AAACCCAAGCGATGAC11,0,unknown,0,unknown
AAACCCAAGGTAAAGG11,0,D6-R3,6,D6
AAACCCAAGTTGAAGT11,0,D6-R1,4,D6
AAACCCACAAATCCCA11,0,D6-R3,6,D6
AAACCCACAACCGTGC11,0,D6-R1,4,D6
...,...,...,...,...
TTTCACAGTCCGGACT-1,1,Nat,9,Nat
TTTCATGCATCCTCAC-1,1,Nat,9,Nat
TTTGATCAGGAGCAAA-1,1,Nat,9,Nat
TTTGGTTAGACGGTTG-1,1,Nat,9,Nat


In [11]:
# Removes cells with unknown sample-id (i.e. cells with ambiguous barcoding)
adata = adata[adata.obs['sample_id']!= 'unknown'] 

In [12]:
# Save adata.
adata.write("2021-06-14_adata.h5ad")

/Users/vickijorgensen/opt/anaconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:1208: ImplicitModificationWarning: Initializing view as actual.
  "Initializing view as actual.", ImplicitModificationWarning
Trying to set attribute `.obs` of view, copying.
... storing 'sample_id' as categorical
Trying to set attribute `.obs` of view, copying.
... storing 'cell_group' as categorical
Trying to set attribute `.var` of view, copying.
... storing 'feature_types' as categorical


In [13]:
%load_ext watermark
%watermark -v -p numpy,pandas,scanpy,anndata,jupyterlab,matplotlib

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.10
IPython 7.22.0

numpy 1.19.4
pandas 1.1.4
scanpy 1.7.2
anndata 0.7.5
jupyterlab 3.0.11
matplotlib 3.3.4
